# Fetching Data from CryptoCompare webpage for News Data for the 10 currencies

Returns latest news
For Sentiment Analysis

In [151]:
# import libraries
import requests
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
from cryptocompare_config import API_Key
import numpy as np
import pymongo
from pymongo import MongoClient
import certifi
from mongo2_config import mongo_uri
import datetime
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
from time import sleep

In [152]:
start_date = datetime.date(year=2020, month=1, day=1,)
print(start_date)

2020-01-01


In [153]:
def convert_epoch(current_date):
#     current_date = current_date.strftime("%Y-%m-%d")
#     current_date_epoch = datetime.datetime.strptime(current_date, "%Y-%m-%d").timestamp()
    current_date_epoch = datetime.datetime(current_date.year, current_date.month,current_date.day,15,59,0).timestamp()
    return current_date_epoch

In [154]:
def api_calling(symbol,epoch_time, date_url_map, current_date):
    url = 'https://min-api.cryptocompare.com/data/v2/news/?'
    url = url + 'categories='
    url = url + symbol + '&excludeCategories=Sponsored'
    url = url + '&lTs='
    url = url + str(epoch_time)
    url = url + '&api_key='+ API_Key
    if not symbol in date_url_map:
        date_url_map[symbol]={}
    else:
        date_url_map[symbol][current_date]=url
#     print(current_date,' : ',url)          

    data_retrieved = requests.get(url).json()    
    data_retrieved_json = data_retrieved['Data']
    news_df = pd.json_normalize(data_retrieved_json)
    if not len(news_df.index) == 0:       
        news_df["Coin_Symbol"] = symbol
        news_df["Rev_time"] = pd.to_datetime(news_df['published_on'],unit='s')
        news_df["Dt_Rev_time"] = news_df['Rev_time'].dt.date
#         print('current_date is ', current_date ,' And the data type is ',  type(current_date))
#         print('f Original ' , news_df[['Coin_Symbol','Rev_time', 'Dt_Rev_time','published_on']])
#         print(news_df.dtypes)
        news_df = news_df[(news_df['Dt_Rev_time']==current_date)]
#         print('f Revised ' , news_df)
    return news_df

In [155]:
print(convert_epoch(start_date))

1577923140.0


In [156]:
import datetime

start_date = datetime.date(year=2020, month=6, day=29)
end_date   = datetime.date(year=2021, month=7,  day=6)
date_url_map = {}

Top_news_df = pd.DataFrame()

current_date = start_date
date_list=[]
api_call_counter = 1
api_call_limit = 50
sleep_time = 1

# Iterating over all dates from start date until end date including end date ("inclusive")
while current_date <= end_date:
    epoch_time = convert_epoch(current_date)
    date_list.append(current_date)
        
    symbols=['BTC','ETH','XRP','USDT','ADA','XMR','XLM','BUSD','LTC','DOGE']
    for symbol in symbols:
#         print(api_call_counter, current_date, symbol)
        if api_call_counter%api_call_limit != 0:
            news_df = api_calling(symbol,epoch_time, date_url_map, current_date)            
            api_call_counter = api_call_counter + 1          
            Top_news_df = Top_news_df.append(news_df, ignore_index = True)
                
        else:
            news_df = api_calling(symbol,epoch_time, date_url_map, current_date)                    
            Top_news_df = Top_news_df.append(news_df, ignore_index = True)
            api_call_counter = 1
            print(f'Going to sleep as {api_call_limit} has reached')
            sleep(sleep_time)
    
    # Advancing current date by one day
    current_date += datetime.timedelta(days=1)
    
# Top_news_df

no_of_days = len(date_list)
print('Total number of days are ' ,no_of_days)

Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to sleep as 50 has reached
Going to s

In [157]:
print(date_list)

[datetime.date(2020, 6, 29), datetime.date(2020, 6, 30), datetime.date(2020, 7, 1), datetime.date(2020, 7, 2), datetime.date(2020, 7, 3), datetime.date(2020, 7, 4), datetime.date(2020, 7, 5), datetime.date(2020, 7, 6), datetime.date(2020, 7, 7), datetime.date(2020, 7, 8), datetime.date(2020, 7, 9), datetime.date(2020, 7, 10), datetime.date(2020, 7, 11), datetime.date(2020, 7, 12), datetime.date(2020, 7, 13), datetime.date(2020, 7, 14), datetime.date(2020, 7, 15), datetime.date(2020, 7, 16), datetime.date(2020, 7, 17), datetime.date(2020, 7, 18), datetime.date(2020, 7, 19), datetime.date(2020, 7, 20), datetime.date(2020, 7, 21), datetime.date(2020, 7, 22), datetime.date(2020, 7, 23), datetime.date(2020, 7, 24), datetime.date(2020, 7, 25), datetime.date(2020, 7, 26), datetime.date(2020, 7, 27), datetime.date(2020, 7, 28), datetime.date(2020, 7, 29), datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), dat

In [158]:
Top_news_df

,id,guid,published_on,imageurl,title,url,source,body,tags,categories,upvotes,downvotes,lang,source_info.name,source_info.lang,source_info.img,Coin_Symbol,Rev_time,Dt_Rev_time
0,17115701,https://www.coindesk.com/index.php?p=489795,1593469399,https://images.cryptocompare.com/news/coindesk...,Digital Currency Guru of $568M Cyber Fraud For...,https://www.coindesk.com/digital-currency-guru...,coindesk,Sergey Medvedev pleaded guilty to racketeering...,News|Fraud|Escrow|Lawsuit|Credit Card Fraud,BTC|Fiat,0,0,EN,CoinDesk,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 22:23:19,2020-06-29
1,17114683,https://dailyhodl.com/?p=102645,1593468320,https://images.cryptocompare.com/news/dailyhod...,XRP Architect David Schwartz Denies Being Bitc...,https://dailyhodl.com/2020/06/29/xrp-architect...,dailyhodl,Ripple&#8217;s chief technology officer David ...,Bitcoin|BTC|News|Ripple|Satoshi Nakamoto|XRP,BTC|XRP|Technology,0,0,EN,The Daily Hodl,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 22:05:20,2020-06-29
2,17113721,https://decrypt.co/?p=34011,1593467166,https://images.cryptocompare.com/news/decrypt/...,How to use DeFi to earn passive income with yo...,https://decrypt.co/34011/how-defi-earn-passive...,decrypt,It’s risky—but it can be done: Andreas Antonop...,Coins|BTC|ETH,BTC|ETH,0,0,EN,Decrypt,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 21:46:06,2020-06-29
3,17113573,https://cointelegraph.com/news/researchers-say...,1593466680,https://images.cryptocompare.com/news/cointele...,Researchers Say New Lightning Network Attack C...,https://cointelegraph.com/news/researchers-say...,cointelegraph,The Lightning Network is susceptible to attack...,Blockchain|Lightning Network|Cybersecurity|Hac...,BTC|Blockchain,0,0,EN,CoinTelegraph,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 21:38:00,2020-06-29
4,17112472,https://cointelegraph.com/news/bitcoin-as-a-to...,1593465480,https://images.cryptocompare.com/news/cointele...,Bitcoin as a tool to fight authoritarian regim...,https://cointelegraph.com/news/bitcoin-as-a-to...,cointelegraph,"Professor of Economics, Sergei Guriev, told Co...",Bitcoin|Blockchain|Russia|Economy|Central Bank,BTC|Blockchain,0,0,EN,CoinTelegraph,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 21:18:00,2020-06-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95831,27010028,https://www.theblockcrypto.com/post/110591,1625598316,https://images.cryptocompare.com/news/default/...,"Fintechs are hungry for DeFi yields, says Comp...",https://www.theblockcrypto.com/post/110591/fin...,theblock,"DeFi and traditional finance are converging, s...",The Block|blockchain|DeFi|Ethereum|fintech,ETH|Blockchain,0,0,EN,TheBlock,EN,https://images.cryptocompare.com/news/default/...,DOGE,2021-07-06 19:05:16,2021-07-06
95832,27010026,https://ambcrypto.com/?p=169149,1625598037,https://images.cryptocompare.com/news/default/...,Why this is crucial to XRP’s trajectory over t...,https://ambcrypto.com/why-this-is-crucial-to-x...,ambcrypto,The global cryptocurrency market noted a bit o...,Altcoins|Analysis|HideCryptopanic|News|News 1|...,XRP|Market|Trading|Altcoin,0,0,EN,AMB Crypto,EN,https://images.cryptocompare.com/news/default/...,DOGE,2021-07-06 19:00:37,2021-07-06
95833,27010027,https://cryptoslate.com/?p=194428,1625598021,https://images.cryptocompare.com/news/default/...,Ripple CTO slams regulators as Barclays blocks...,https://cryptoslate.com/ripple-cto-slams-regul...,cryptoslate,Barclays UK customers have been blocked from t...,Banking|Exchanges|Regulation|xrp,Exchange|XRP|Regulation|Fiat,0,0,EN,CryptoSlate,EN,https://images.cryptocompare.com/news/default/...,DOGE,2021-07-06 19:00:21,2021-07-06
95834,27010025,https://www.livebitcoinnews.com/?p=65282,1625598011,https://images.cryptocompare.com/news/default/...,Crypto Head: The U.S. Is the “Most-Ready” Nati...,https://www.livebitcoinnews.com/crypto-head-th...,livebitcoinnews,A

In [159]:
Top_news_df.shape[0]

95836

In [160]:
date_url_map

{'BTC': {datetime.date(2020, 6, 30): 'https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&excludeCategories=Sponsored&lTs=1593557940.0&api_key=01b113df13b65ddc8d12dfcfeaca9190479087fc9ffc46756e84cee398ba7699',
  datetime.date(2020, 7, 1): 'https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&excludeCategories=Sponsored&lTs=1593644340.0&api_key=01b113df13b65ddc8d12dfcfeaca9190479087fc9ffc46756e84cee398ba7699',
  datetime.date(2020, 7, 2): 'https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&excludeCategories=Sponsored&lTs=1593730740.0&api_key=01b113df13b65ddc8d12dfcfeaca9190479087fc9ffc46756e84cee398ba7699',
  datetime.date(2020, 7, 3): 'https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&excludeCategories=Sponsored&lTs=1593817140.0&api_key=01b113df13b65ddc8d12dfcfeaca9190479087fc9ffc46756e84cee398ba7699',
  datetime.date(2020, 7, 4): 'https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&excludeCategories=Sponsored&lTs=1593903540.0&ap

In [161]:
Top_news_2021_07_06_df = Top_news_df
Top_news_2021_07_06_df

,id,guid,published_on,imageurl,title,url,source,body,tags,categories,upvotes,downvotes,lang,source_info.name,source_info.lang,source_info.img,Coin_Symbol,Rev_time,Dt_Rev_time
0,17115701,https://www.coindesk.com/index.php?p=489795,1593469399,https://images.cryptocompare.com/news/coindesk...,Digital Currency Guru of $568M Cyber Fraud For...,https://www.coindesk.com/digital-currency-guru...,coindesk,Sergey Medvedev pleaded guilty to racketeering...,News|Fraud|Escrow|Lawsuit|Credit Card Fraud,BTC|Fiat,0,0,EN,CoinDesk,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 22:23:19,2020-06-29
1,17114683,https://dailyhodl.com/?p=102645,1593468320,https://images.cryptocompare.com/news/dailyhod...,XRP Architect David Schwartz Denies Being Bitc...,https://dailyhodl.com/2020/06/29/xrp-architect...,dailyhodl,Ripple&#8217;s chief technology officer David ...,Bitcoin|BTC|News|Ripple|Satoshi Nakamoto|XRP,BTC|XRP|Technology,0,0,EN,The Daily Hodl,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 22:05:20,2020-06-29
2,17113721,https://decrypt.co/?p=34011,1593467166,https://images.cryptocompare.com/news/decrypt/...,How to use DeFi to earn passive income with yo...,https://decrypt.co/34011/how-defi-earn-passive...,decrypt,It’s risky—but it can be done: Andreas Antonop...,Coins|BTC|ETH,BTC|ETH,0,0,EN,Decrypt,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 21:46:06,2020-06-29
3,17113573,https://cointelegraph.com/news/researchers-say...,1593466680,https://images.cryptocompare.com/news/cointele...,Researchers Say New Lightning Network Attack C...,https://cointelegraph.com/news/researchers-say...,cointelegraph,The Lightning Network is susceptible to attack...,Blockchain|Lightning Network|Cybersecurity|Hac...,BTC|Blockchain,0,0,EN,CoinTelegraph,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 21:38:00,2020-06-29
4,17112472,https://cointelegraph.com/news/bitcoin-as-a-to...,1593465480,https://images.cryptocompare.com/news/cointele...,Bitcoin as a tool to fight authoritarian regim...,https://cointelegraph.com/news/bitcoin-as-a-to...,cointelegraph,"Professor of Economics, Sergei Guriev, told Co...",Bitcoin|Blockchain|Russia|Economy|Central Bank,BTC|Blockchain,0,0,EN,CoinTelegraph,EN,https://images.cryptocompare.com/news/default/...,BTC,2020-06-29 21:18:00,2020-06-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95831,27010028,https://www.theblockcrypto.com/post/110591,1625598316,https://images.cryptocompare.com/news/default/...,"Fintechs are hungry for DeFi yields, says Comp...",https://www.theblockcrypto.com/post/110591/fin...,theblock,"DeFi and traditional finance are converging, s...",The Block|blockchain|DeFi|Ethereum|fintech,ETH|Blockchain,0,0,EN,TheBlock,EN,https://images.cryptocompare.com/news/default/...,DOGE,2021-07-06 19:05:16,2021-07-06
95832,27010026,https://ambcrypto.com/?p=169149,1625598037,https://images.cryptocompare.com/news/default/...,Why this is crucial to XRP’s trajectory over t...,https://ambcrypto.com/why-this-is-crucial-to-x...,ambcrypto,The global cryptocurrency market noted a bit o...,Altcoins|Analysis|HideCryptopanic|News|News 1|...,XRP|Market|Trading|Altcoin,0,0,EN,AMB Crypto,EN,https://images.cryptocompare.com/news/default/...,DOGE,2021-07-06 19:00:37,2021-07-06
95833,27010027,https://cryptoslate.com/?p=194428,1625598021,https://images.cryptocompare.com/news/default/...,Ripple CTO slams regulators as Barclays blocks...,https://cryptoslate.com/ripple-cto-slams-regul...,cryptoslate,Barclays UK customers have been blocked from t...,Banking|Exchanges|Regulation|xrp,Exchange|XRP|Regulation|Fiat,0,0,EN,CryptoSlate,EN,https://images.cryptocompare.com/news/default/...,DOGE,2021-07-06 19:00:21,2021-07-06
95834,27010025,https://www.livebitcoinnews.com/?p=65282,1625598011,https://images.cryptocompare.com/news/default/...,Crypto Head: The U.S. Is the “Most-Ready” Nati...,https://www.livebitcoinnews.com/crypto-head-th...,livebitcoinnews,A

# Exporting data to csv and Uploading on MongoDb

In [162]:
# exporting the dataframe to csv
Top_news_2021_07_06_df.to_csv('Top_news_2020to2021_07_06(5year).csv')

In [114]:
print(Top_news_2020_03_03_df.dtypes)

id                          object
guid                        object
published_on                 int64
imageurl                    object
title                       object
url                         object
source                      object
body                        object
tags                        object
categories                  object
upvotes                     object
downvotes                   object
lang                        object
source_info.name            object
source_info.lang            object
source_info.img             object
Coin_Symbol                 object
Rev_time            datetime64[ns]
Dt_Rev_time                 object
dtype: object


In [78]:
# # Read in mongodb server location as client
# client = MongoClient(mongo_uri,tlsCAFile=certifi.where())

In [79]:
# # Find Databases
# client.list_database_names()

['Kaggle_data', 'Latest_Cryptodata', 'Latest_StockMarket', 'admin', 'local']

In [82]:
# # connecting to the db on MongoDb and creating collection in the db
# db = client['Latest_Cryptodata']
# collection = db['Top_news_2016to2019_08_03(5year)']

In [83]:
# Top_news_2019_08_03_df.reset_index(inplace=True)
# Top_news_2019_08_03_df_dict = Top_news_2019_08_03_df.to_dict("records")

In [ ]:
# # Insert collection
# collection.insert_many(Top_news_2019_08_03_df_dict)